# OMI

The files of OMI are in he5 format. We are taking monthly averages to make the data compatible to POLDER. Since, POLDER data was in monthly averages for 2006. 



In [1]:
import rasterio 
import numpy as np
import matplotlib.pyplot as plt
import h5py
%matplotlib notebook
%qtconsole

In [2]:
import numpy as np
import h5py
import glob
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from matplotlib.pylab import plt

In [3]:
import h5py
filename = "DATA/OMId3/omilevel3/OMI-Aura_L3-OMAEROe_2006m0102_v003-2011m1109t085136.he5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])

Keys: <KeysViewHDF5 ['HDFEOS', 'HDFEOS INFORMATION']>


In [4]:
f = h5py.File(filename, mode='r')
for name in f:
    print(name)

HDFEOS
HDFEOS INFORMATION


In [5]:
f.visit(lambda x: print(x))

HDFEOS
HDFEOS/ADDITIONAL
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES
HDFEOS/GRIDS
HDFEOS/GRIDS/ColumnAmountAerosol
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AbsorbingAerosolOpticalThicknessMW
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolModelMW
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolOpticalThicknessMW
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolOpticalThicknessPassedThresholdMean
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolOpticalThicknessPassedThresholdStd
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/Latitude
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/Longitude
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/SingleScatteringAlbedoMW
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/SingleScatteringAlbedoPassedThresholdMean
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/SingleScatteringAlbedoPassedThresholdStd
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/SolarZenithAngle
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/Terra

In [6]:
f = h5py.File(filename, mode='r')

f.visit(lambda x: print(x))

HDFEOS
HDFEOS/ADDITIONAL
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES
HDFEOS/GRIDS
HDFEOS/GRIDS/ColumnAmountAerosol
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AbsorbingAerosolOpticalThicknessMW
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolModelMW
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolOpticalThicknessMW
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolOpticalThicknessPassedThresholdMean
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolOpticalThicknessPassedThresholdStd
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/Latitude
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/Longitude
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/SingleScatteringAlbedoMW
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/SingleScatteringAlbedoPassedThresholdMean
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/SingleScatteringAlbedoPassedThresholdStd
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/SolarZenithAngle
HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/Terra

In [12]:

Aerosolmodel = f['HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/AerosolModelMW']

In [25]:
print(Aerosolmodel.len)

<bound method Dataset.len of <HDF5 dataset "AerosolModelMW": shape (720, 1440), type "<u2">>


In [14]:
lat = f['HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/Latitude']
lon = f['HDFEOS/GRIDS/ColumnAmountAerosol/Data Fields/Longitude']

In [15]:
lat

<HDF5 dataset "Latitude": shape (720, 1440), type "<f4">

In [16]:
so2 = Aerosolmodel

In [17]:
so2

<HDF5 dataset "AerosolModelMW": shape (720, 1440), type "<u2">

In [18]:
for a in so2.attrs:
    print(a, so2.attrs[a])
    
# Some of these are floats, others are strings, extracting values is difference depending on which:
nodata = so2.attrs['MissingValue'][0]
units = so2.attrs['Units'].decode()
titlestr = so2.attrs['Title'].decode()
titlestr = so2.attrs['UniqueFieldDefinition'].decode()

MissingValue [65535]
Offset [0.]
ScaleFactor [1.]
Title b'Aerosol model indicator for best fit aerosol model derived with the Multi-Wavelength method'
UniqueFieldDefinition b'OMI-Specific'
Units b'NoUnits'
_FillValue [65535]


In [19]:
FILL_VALUE = -1,2676506e+30

def monthly_no2(files, filter_to_nan = 14.5): 
    '''
    
    Return monthly averaged $NO_2$ matrix from  a list of files
    
    
    
    '''
    
    arr = np.empty(len(files), lat.shape[0], lon.shape[0], dtype = float) 
    p = 'ColumnAmountNO2TropCloudScreened'
    for ix, file in enumerate(files): 
        with h5py.File(file, 'r') as f:
            data = f['HDFEOS']['GRIDS']['ColumnAmountNO2']['Data Fields'][p][:]
            data[(data == FILL_VALUE) | (data <= 0)] = np.nan
            arr[ix] = data 
            
    average_monthly = np.log10(np.nanmean(arr,axis = 0))
    average_monthly[average_monthly < filter_to_nan] = np.nan
    return average_monthly


In [20]:
average_monthly = monthly_no2(files)

NameError: name 'files' is not defined

In [21]:
# Making the countries map 
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world = world[['continent', 'geometry']]

continents = world.dissolve(by='continent')
continents = world.dissolve(by='continent')
continents = continents[~continents.index.isin(['Seven seas (open ocean)', 'Antarctica'])]
continents = continents.geometry.explode().reset_index()

In [22]:
continents


,continent,level_1,geometry
0,Africa,0,"POLYGON ((32.83012 -26.74219, 32.58026 -27.470..."
1,Africa,1,"POLYGON ((49.54352 -12.46983, 49.80898 -12.895..."
2,Asia,0,"POLYGON ((120.29501 -10.25865, 118.96781 -9.55..."
3,Asia,1,"POLYGON ((108.48685 -6.42198, 108.62348 -6.777..."
4,Asia,2,"POLYGON ((134.21013 -6.89524, 134.11278 -6.142..."
...,...,...,...
120,Oceania,17,"POLYGON ((162.11902 -10.48272, 162.39865 -10.8..."
121,Oceania,18,"POLYGON ((141.00021 -2.60015, 142.73525 -3.289..."
122,South America,0,"POLYGON ((-66.95992 -54.89681, -67.29103 -55.3..."
123,South America,1,"POLYGON ((-61.20000 -51.85000, -60.00000 -51.2..."


In [11]:

data = so2.value
nodata = so2.attrs['MissingValue']
data[so2.value == nodata] = np.nan

AttributeError: 'Dataset' object has no attribute 'value'

In [ ]:
# Import Python packages (tools for working with geospatial data)
import rasterio 
import numpy as np
import matplotlib.pyplot as plt
import h5py
%matplotlib notebook
%qtconsole

plt.figure(figsize=(8,4))
plt.imshow(data)
cb = plt.colorbar()
cb.set_label(units)
plt.title(filename)

In [26]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import h5py

In [28]:
# Open file.
filename = "DATA/OMId3/omilevel3/OMI-Aura_L3-OMAEROe_2006m0102_v003-2011m1109t085136.he5"
DATAFIELD_NAME = '/HDFEOS/GRIDS/OMI Column Amount O3/Data Fields/ColumnAmountO3'
with h5py.File(filename, mode='r') as f:
# List available datasets.
    print f.keys()

# Read dataset.
    dset = f[DATAFIELD_NAME]
    data = dset[:]

# Handle fill value.
    data[data == dset.fillvalue] = np.nan
    data = np.ma.masked_where(np.isnan(data), data)

# Get attributes needed for the plot.
# String attributes actually come in as the bytes type and should
# be decoded to UTF-8 (python3).
    title = dset.attrs['Title'].decode()
    units = dset.attrs['Units'].decode()

SyntaxError: invalid syntax (<ipython-input-28-354f2f9aa4fe>, line 6)